In [1]:
import os
import sys
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
sys.path.insert(0, '../scripts')

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.layers import Dense, Activation, Dropout, LSTM, Bidirectional
from mfcc_feature_extraction import MFCCFeatureExtraction
from load_file import LoadFile

In [3]:
tracks, labels = LoadFile('../assets/csv/test.csv').__getitem__()
test_features, test_labels = MFCCFeatureExtraction(tracks, labels).featuredata()
tracks, labels = LoadFile('../assets/csv/dev.csv').__getitem__()
dev_features, dev_labels = MFCCFeatureExtraction(tracks, labels).featuredata()
tracks, labels = LoadFile('../assets/csv/train.csv').__getitem__()
train_features, train_labels = MFCCFeatureExtraction(tracks, labels).featuredata()

train_features = tf.expand_dims(train_features, axis = -1)
test_features = tf.expand_dims(test_features, axis = -1)
dev_features = tf.expand_dims(dev_features, axis = -1)

In [4]:
dense_layer =  tf.keras.Sequential([
                                      Dense(128, input_shape =(40,) ),
                                      Activation('relu'),
                                      Dropout(0.5),

                                      Dense(256),
                                      Activation('relu'),
                                      Dropout(0.5),
                                    
                                      Dense(128),
                                      Activation('relu'),
                                      Dropout(0.5),
                                    
                                      Dense(5),
                                      Activation('softmax')
])

input = Input(shape = (20,1))
x = Bidirectional(LSTM(20))(input)
output = dense_layer(x)

model = tf.keras.Model(inputs = input, outputs = output)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 1)]           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 40)                3520      
_________________________________________________________________
sequential (Sequential)      (None, 5)                 71813     
Total params: 75,333
Trainable params: 75,333
Non-trainable params: 0
_________________________________________________________________


In [8]:
from datetime import datetime
from tensorflow.keras.callbacks import ModelCheckpoint

model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
num_epochs = 1000
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath = '../Checkpoints/audio_blstm.hdf5', verbose = 1, save_best_only = True)

In [9]:
#LSTM and Dense Layer
start = datetime.now()
model.fit(train_features, train_labels,batch_size = num_batch_size, epochs = num_epochs, validation_data = (dev_features, dev_labels), callbacks = [checkpointer])
end = datetime.now()

print("Training completed in time:", end - start)

Epoch 1/1000
132/141 [===========================>..] - ETA: 0s - loss: 0.1967 - accuracy: 0.9167
Epoch 00001: val_loss improved from inf to 0.25254, saving model to ../Checkpoints/audio_blstm.hdf5
141/141 [==============================] - 1s 8ms/step - loss: 0.1953 - accuracy: 0.9167 - val_loss: 0.2525 - val_accuracy: 0.9160
Epoch 2/1000
131/141 [==========================>...] - ETA: 0s - loss: 0.1888 - accuracy: 0.9237
Epoch 00002: val_loss did not improve from 0.25254
141/141 [==============================] - 1s 5ms/step - loss: 0.1910 - accuracy: 0.9231 - val_loss: 0.3110 - val_accuracy: 0.9000
Epoch 3/1000
136/141 [===========================>..] - ETA: 0s - loss: 0.2088 - accuracy: 0.9148
Epoch 00003: val_loss did not improve from 0.25254
141/141 [==============================] - 1s 5ms/step - loss: 0.2097 - accuracy: 0.9138 - val_loss: 0.2837 - val_accuracy: 0.9000
Epoch 4/1000
133/141 [===========================>..] - ETA: 0s - loss: 0.2025 - accuracy: 0.9213
Epoch 00004: 

In [10]:
# LSTM and dense layer
model.evaluate(test_features, test_labels, verbose = 1)

8/8 [==============================] - 0s 1ms/step - loss: 0.6610 - accuracy: 0.8960


[0.6609980463981628, 0.8960000276565552]